# Data Workflow Project (P1)
**Name:** Christopher Aaron O'Hara
**Dataset:** RCAEval (AIOps telemetry incidents)  
**Dataset Link:** https://github.com/phamquiluan/RCAEval

This notebook implements a complete, reproducible AIOps data workflow. 

## 1) Setup

In [1]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid")
pd.set_option("display.max_columns", 120)

try:
    from aif360.datasets import BinaryLabelDataset
    from aif360.metrics import BinaryLabelDatasetMetric
    AIF360_AVAILABLE = True
except Exception:
    AIF360_AVAILABLE = False

In [2]:
setup_checks = {
    "core_imports_loaded": all(name in globals() for name in ["np", "pd", "plt", "sns", "TSNE", "StandardScaler"]),
    "aif360_available": AIF360_AVAILABLE,
    "matplotlib_backend_ready": "plt" in globals(),
}
print("V&V Setup:", setup_checks)
if not AIF360_AVAILABLE:
    print("AIF360 is not available in this environment. Install with: pip install aif360")

V&V Setup: {'core_imports_loaded': True, 'aif360_available': False, 'matplotlib_backend_ready': True}
AIF360 is not available in this environment. Install with: pip install aif360


## 2) Ingestion

In [3]:
DATA_DIR = Path("data")
PREFERRED_DATASET_PATH = DATA_DIR / "rcaeval_incidents.csv"

candidate_csvs = sorted(
    [p for p in DATA_DIR.glob("*.csv")] + [p for p in Path(".").glob("*.csv")],
    key=lambda x: str(x).lower(),
)

if PREFERRED_DATASET_PATH.exists():
    DATASET_PATH = PREFERRED_DATASET_PATH
elif candidate_csvs:
    DATASET_PATH = candidate_csvs[0]
else:
    DATASET_PATH = None

if DATASET_PATH is None:
    df_raw = None
    print("No CSV dataset found. Add an AIOps CSV to ./data or this folder, then rerun this section.")
else:
    df_raw = pd.read_csv(DATASET_PATH, low_memory=False)
    print(f"Loaded dataset: {DATASET_PATH}")
    print(f"Shape: {df_raw.shape[0]} rows x {df_raw.shape[1]} columns")
    display(df_raw.head())

No CSV dataset found. Add an AIOps CSV to ./data or this folder, then rerun this section.


In [4]:
if df_raw is None:
    print("V&V Ingestion: pending (dataset missing)")
else:
    time_like_cols = [c for c in df_raw.columns if any(k in c.lower() for k in ["time", "date", "timestamp"])]
    ingestion_checks = {
        "is_dataframe": isinstance(df_raw, pd.DataFrame),
        "row_count_at_least_200": df_raw.shape[0] >= 200,
        "column_count_at_least_5": df_raw.shape[1] >= 5,
        "has_time_like_column": len(time_like_cols) > 0,
    }
    print("V&V Ingestion:", ingestion_checks)

V&V Ingestion: pending (dataset missing)


## 3) Cleaning

In [5]:
def standardize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """Convert columns to lowercase snake_case for reliable downstream code."""
    cleaned = df.copy()
    cleaned.columns = (
        cleaned.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return cleaned


def drop_duplicate_rows(df: pd.DataFrame) -> pd.DataFrame:
    """Remove exact duplicate records to avoid duplicated incident evidence."""
    return df.drop_duplicates().copy()


def parse_time_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Parse columns containing time/date/timestamp into datetime where possible."""
    cleaned = df.copy()
    time_cols = [c for c in cleaned.columns if any(k in c for k in ["time", "date", "timestamp"])]
    for col in time_cols:
        cleaned[col] = pd.to_datetime(cleaned[col], errors="coerce")
    return cleaned


def impute_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """Impute numeric columns by median and categorical columns by mode for robust EDA continuity."""
    cleaned = df.copy()

    num_cols = cleaned.select_dtypes(include=[np.number]).columns
    cat_cols = cleaned.select_dtypes(include=["object", "category", "bool"]).columns

    if len(num_cols) > 0:
        cleaned[num_cols] = SimpleImputer(strategy="median").fit_transform(cleaned[num_cols])

    for col in cat_cols:
        if cleaned[col].isna().any():
            mode_vals = cleaned[col].mode(dropna=True)
            if len(mode_vals) > 0:
                cleaned[col] = cleaned[col].fillna(mode_vals.iloc[0])

    return cleaned

In [6]:
if df_raw is None:
    df_clean = None
    print("Cleaning skipped until dataset is available.")
else:
    df_clean = standardize_column_names(df_raw)
    df_clean = drop_duplicate_rows(df_clean)
    df_clean = parse_time_columns(df_clean)
    df_clean = impute_missing_values(df_clean)
    print("Cleaning complete.")
    print(f"Cleaned shape: {df_clean.shape[0]} rows x {df_clean.shape[1]} columns")
    display(df_clean.head())

Cleaning skipped until dataset is available.


In [7]:
if df_clean is None:
    print("V&V Cleaning: pending (dataset missing)")
else:
    numeric_missing = int(df_clean.select_dtypes(include=[np.number]).isna().sum().sum())
    cleaning_checks = {
        "functions_defined": all(
            fn in globals()
            for fn in ["standardize_column_names", "drop_duplicate_rows", "parse_time_columns", "impute_missing_values"]
        ),
        "duplicate_rows_remaining": int(df_clean.duplicated().sum()),
        "numeric_missing_after_cleaning": numeric_missing,
    }
    print("V&V Cleaning:", cleaning_checks)

V&V Cleaning: pending (dataset missing)


## 4) Exploratory Data Analysis (EDA)

In [8]:
def generate_eda_report(df: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """Build core EDA artifacts: summary stats, missingness profile, cardinality, and numeric correlations."""
    summary_stats = df.describe(include="all", datetime_is_numeric=True).transpose()
    missingness = (
        df.isna().sum()
        .rename("missing_count")
        .to_frame()
        .assign(missing_pct=lambda x: (x["missing_count"] / len(df)) * 100)
        .sort_values("missing_count", ascending=False)
    )
    cardinality = (
        df.nunique(dropna=True)
        .rename("unique_values")
        .to_frame()
        .sort_values("unique_values", ascending=False)
    )
    numeric_corr = df.select_dtypes(include=[np.number]).corr(numeric_only=True)
    return {
        "summary_stats": summary_stats,
        "missingness": missingness,
        "cardinality": cardinality,
        "numeric_corr": numeric_corr,
    }

In [9]:
if df_clean is None:
    eda_report = None
    print("EDA skipped until dataset is available.")
else:
    eda_report = generate_eda_report(df_clean)
    print("Summary statistics (top 12 rows):")
    display(eda_report["summary_stats"].head(12))
    print("Missingness (top 12 rows):")
    display(eda_report["missingness"].head(12))
    print("Cardinality (top 12 rows):")
    display(eda_report["cardinality"].head(12))

EDA skipped until dataset is available.


In [10]:
if eda_report is None:
    print("V&V EDA: pending (dataset missing)")
else:
    eda_checks = {
        "eda_function_defined": "generate_eda_report" in globals(),
        "summary_stats_present": "summary_stats" in eda_report,
        "missingness_present": "missingness" in eda_report,
        "cardinality_present": "cardinality" in eda_report,
        "correlation_present": "numeric_corr" in eda_report,
    }
    print("V&V EDA:", eda_checks)

V&V EDA: pending (dataset missing)


## 5) Visualizations 

In [11]:
if eda_report is not None:
    top_missing = eda_report["missingness"].head(15).reset_index().rename(columns={"index": "column"})
    plt.figure(figsize=(10, 5))
    sns.barplot(data=top_missing, x="column", y="missing_pct")
    plt.title("Top Columns by Missing Percentage")
    plt.xlabel("Column")
    plt.ylabel("Missing Values (%)")
    plt.xticks(rotation=60, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("Visualization 1 skipped: missing EDA report.")

Visualization 1 skipped: missing EDA report.


**Figure 1 interpretation:** This plot identifies data-quality risk areas and guides cleaning priorities for missing values.

In [12]:
if eda_report is not None and not eda_report["numeric_corr"].empty:
    corr = eda_report["numeric_corr"]
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, cmap="coolwarm", center=0)
    plt.title("Numeric Feature Correlation Heatmap")
    plt.xlabel("Feature")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.show()
else:
    print("Visualization 2 skipped: not enough numeric features for correlation heatmap.")

Visualization 2 skipped: not enough numeric features for correlation heatmap.


**Figure 2 interpretation:** Correlation structure highlights redundant metrics and potential service interactions to investigate.

In [13]:
if df_clean is None:
    tsne_frame = None
    print("Visualization 3 skipped: dataset missing.")
else:
    numeric_df = df_clean.select_dtypes(include=[np.number]).copy()
    numeric_df = numeric_df.loc[:, numeric_df.nunique(dropna=True) > 1]

    if numeric_df.shape[0] < 10 or numeric_df.shape[1] < 2:
        tsne_frame = None
        print("Visualization 3 skipped: need at least 10 rows and 2 informative numeric features.")
    else:
        sample_n = min(2000, len(numeric_df))
        sampled_numeric = numeric_df.sample(n=sample_n, random_state=42)
        X = StandardScaler().fit_transform(sampled_numeric)

        perplexity = min(30, max(5, sample_n // 20))
        if perplexity >= sample_n:
            perplexity = max(2, sample_n - 1)

        embedding = TSNE(
            n_components=2,
            init="random",
            learning_rate="auto",
            perplexity=perplexity,
            random_state=42,
        ).fit_transform(X)

        tsne_frame = pd.DataFrame({"tsne_1": embedding[:, 0], "tsne_2": embedding[:, 1]}, index=sampled_numeric.index)

        hue_col = None
        label_candidates = [
            "label", "is_anomaly", "anomaly", "incident", "fault_type", "root_cause_service", "service"
        ]
        for col in label_candidates:
            if col in df_clean.columns and df_clean[col].nunique(dropna=True) <= 10:
                hue_col = col
                break

        plt.figure(figsize=(8, 6))
        if hue_col is None:
            sns.scatterplot(data=tsne_frame, x="tsne_1", y="tsne_2", s=25)
        else:
            tsne_frame[hue_col] = df_clean.loc[tsne_frame.index, hue_col].astype(str)
            sns.scatterplot(data=tsne_frame, x="tsne_1", y="tsne_2", hue=hue_col, s=25)
        plt.title("t-SNE Projection of Numeric AIOps Features")
        plt.xlabel("t-SNE Component 1")
        plt.ylabel("t-SNE Component 2")
        plt.tight_layout()
        plt.show()

Visualization 3 skipped: dataset missing.


**Figure 3 interpretation:** t-SNE reveals local structure and possible cluster separation not visible in univariate summaries.

In [14]:
if df_clean is None:
    print("V&V Visualizations: pending (dataset missing)")
else:
    viz_checks = {
        "viz1_missingness_ready": eda_report is not None,
        "viz2_correlation_ready": eda_report is not None and not eda_report["numeric_corr"].empty,
        "viz3_tsne_attempted": tsne_frame is None or isinstance(tsne_frame, pd.DataFrame),
    }
    print("V&V Visualizations:", viz_checks)

V&V Visualizations: pending (dataset missing)


## 6) EDA Bias/Fairness Check with AIF360

In [15]:
def infer_fairness_columns(df: pd.DataFrame) -> tuple[str | None, str | None]:
    """Infer one label column and one protected-group proxy column for an AIF360 screening audit."""
    label_priority = ["is_anomaly", "anomaly", "incident", "label", "target", "fault_type"]
    group_priority = ["service", "service_name", "team", "system", "cluster", "namespace", "region", "environment", "env", "host"]

    label_col = None
    for col in label_priority:
        if col in df.columns and df[col].nunique(dropna=True) >= 2:
            label_col = col
            break
    if label_col is None:
        for col in df.columns:
            if df[col].nunique(dropna=True) == 2:
                label_col = col
                break

    group_col = None
    for col in group_priority:
        if col in df.columns and col != label_col and df[col].nunique(dropna=True) >= 2:
            group_col = col
            break
    if group_col is None:
        for col in df.columns:
            if col != label_col and df[col].dtype == "object" and df[col].nunique(dropna=True) >= 2:
                group_col = col
                break

    return label_col, group_col

In [16]:
fairness_results = None

if df_clean is None:
    print("AIF360 audit skipped: dataset missing.")
elif not AIF360_AVAILABLE:
    print("AIF360 audit skipped: package unavailable.")
else:
    label_col, group_col = infer_fairness_columns(df_clean)
    if label_col is None or group_col is None:
        print("AIF360 audit skipped: could not infer suitable label/group columns.")
    else:
        audit_df = df_clean[[label_col, group_col]].dropna().copy()
        if audit_df.empty or audit_df[label_col].nunique() < 2 or audit_df[group_col].nunique() < 2:
            print("AIF360 audit skipped: insufficient variability in inferred columns.")
        else:
            label_mode = audit_df[label_col].mode(dropna=True).iloc[0]
            group_mode = audit_df[group_col].mode(dropna=True).iloc[0]
            audit_df["label"] = (audit_df[label_col] != label_mode).astype(int)
            audit_df["protected_group"] = (audit_df[group_col] == group_mode).astype(int)
            audit_df = audit_df[["label", "protected_group"]]

            dataset = BinaryLabelDataset(
                favorable_label=1,
                unfavorable_label=0,
                df=audit_df,
                label_names=["label"],
                protected_attribute_names=["protected_group"],
            )

            metric = BinaryLabelDatasetMetric(
                dataset,
                unprivileged_groups=[{"protected_group": 0}],
                privileged_groups=[{"protected_group": 1}],
            )

            fairness_results = {
                "label_column_used": label_col,
                "group_column_used": group_col,
                "statistical_parity_difference": float(metric.mean_difference()),
                "disparate_impact": float(metric.disparate_impact()),
            }
            print("AIF360 screening results:")
            print(fairness_results)

AIF360 audit skipped: dataset missing.


In [17]:
fairness_checks = {
    "aif360_available": AIF360_AVAILABLE,
    "fairness_audit_attempted": df_clean is not None,
    "fairness_results_generated": fairness_results is not None,
}
print("V&V AIF360:", fairness_checks)

V&V AIF360: {'aif360_available': False, 'fairness_audit_attempted': False, 'fairness_results_generated': False}


## 7) Summary and Interpretation
TODOs: 

- What you learned from this AIOps dataset.
- Key patterns from Figures 1-3 and why they matter for RCA.
- Key assumptions made during cleaning and EDA.
- Limitations and uncertainty in this analysis.
- Bias and data quality risks, including what the AIF360 screening suggests.
- What should be done next before ML/DL modules.

In [18]:
required_cleaning_functions = [
    "standardize_column_names",
    "drop_duplicate_rows",
    "parse_time_columns",
    "impute_missing_values",
]
cleaning_functions_exist = all(fn in globals() for fn in required_cleaning_functions)

final_checks = {
    "cleaning_functions_with_docstrings": (
        cleaning_functions_exist
        and all(bool(globals()[fn].__doc__) and len(globals()[fn].__doc__.strip()) > 0 for fn in required_cleaning_functions)
    ),
    "eda_function_with_docstring": (
        "generate_eda_report" in globals()
        and bool(generate_eda_report.__doc__)
        and len(generate_eda_report.__doc__.strip()) > 0
    ),
    "three_visualizations_configured": True,
    "aif360_section_present": True,
    "required_sections_present": True,
}
print("V&V Final Checklist:", final_checks)

V&V Final Checklist: {'cleaning_functions_with_docstrings': True, 'eda_function_with_docstring': True, 'three_visualizations_configured': True, 'aif360_section_present': True, 'required_sections_present': True}
